### Notebook to parse text files to produce cleaned text of RAD decisions

Sean Rehaag

License: Creative Commons Attribution-NonCommercial 4.0 International (CC BY-NC 4.0). 

Dataset & Code to be cited as:

Sean Rehaag, "Refugee Appeal Division Bulk Decisions Dataset" (2023), online: Refugee Law Laboratory <https://refugeelab.ca/bulk-data/rad/>.

Notes:

(1) Data Source: Immigration and Refugee Board. In the Fall of 2022, the IRB added the Refugee Law Laboratory to their email distribution list for legal publishers of RAD decisions. The RLL therefore receives new RAD cases as they are released for publication by the IRB. Also, in the fall of 2022 the Immigration and Refugee Board provided the RLL with a full backlog of approximately 116k published decisions from all divisions (RAD, RPD, ID, IAD). 

(2) Unofficial Data: The data are unofficial reproductions. For official versions, please contact the Immigration and Refugee Board. 

(3) Non-Affiliation / Endorsement: The data has been collected and reproduced without any affiliation or endorsement from the Immigration and Refugee Board.

(4) Non-Commerical Use: As indicated in the license, data may be used for non-commercial use (with attribution) only. For commercial use, please contact the Immigration and Refugee Board. 

(5) Accuracy: Data was collected and processed programmatically for the purposes of academic research. While we make best efforts to ensure accuracy, data gathering of this kind inevitably involves errors. As such the data should be viewed as preliminary information aimed to prompt further research and discussion, rather than as definitive information.

Acknowledgements: Thanks to Rafael Dolores for coding the parsing scripts.


# Installing Libraries

In [1]:
#!pip install langdetect
#!pip install regex

# Importing Libraries

In [2]:
import os
import regex as re 
import pandas as pd
from datetime import datetime
from langdetect import detect
from difflib import get_close_matches
import json
from tqdm import tqdm

## Declaring Constant
Here, we specify the directories containing our data files.

In [3]:
DATA_DIRS = ["../RAD Decisions TEXT", "../IRB Decisions - Initial Request - TEXT"]

# # For SR:
# DATA_DIRS = ["d:/RAD Decisions TEXT/", "d:/IRB Decisions - Initial Request - TEXT/"]



## Language Detection
This function determines the language of a given text.

In [4]:
def detect_language(text):
    try:
        return detect(text)
    except:
        return None

## Decision Maker Extraction
This function searches the given file for the decision maker using regular expressions.

In [5]:
def extract_decision_maker(content):
    patterns = [
        # String in line immediately after 'Panel' and before 'Tribunal', allowing tabs and spaces
        r"^Panel\s*([^\n]+?)\s*\n\s*Tribunal\s*$",  
      
        # String in line immediately after 'Tribunal' and before 'Panel', allowing tabs and spaces
        r"^Tribunal\s*([^\n]+?)\s*\n\s*Panel\s*$",
        # String in line immediately after 'Tribunal' and followed by another 'Tribunal', allowing tabs and spaces
        r"^Tribunal\s*([^\n]+?)\s*\n\s*Tribunal\s*$"
    ]

    for pattern in patterns:
        # Use re.MULTILINE to allow ^ and $ to match the start and end of each line
        match = re.search(pattern, content, re.IGNORECASE | re.MULTILINE)
        if match:
            captured = match.group(1).strip()
            # Check if captured group ends with 'Tribunal' or 'Panel' and exclude it
            if not captured.endswith("Tribunal") and not captured.endswith("Panel"):
                return captured
    return None



## Regular Expression Detector
Functions to parse the date from text files while accounting for several different formats

In [6]:
def match_date_patterns(content):
    patterns = {
        "custom": (r"Date (?:of decision|de la décision)\s*\n\s*([A-Za-z]+)\s+(\d{1,2})\.\s*(\d{4})", lambda m: [m.group(1), m.group(2), m.group(3)]),
        "primary": (r"Date (?:of decision|de la décision)\s*(?:Le )?\s*((?:(?:\d{1,2}|1er)\s+[\w]+\s*,?\s*\d{1,4})|\w+\s+\d{1,2}(?:st|nd|rd|th)?\s*,?\s*\d{1,4}|\d{1,2}-\d{1,2}-\d{1,4})", lambda m: m.group(1).replace(',', '').split()),
        "original_decision": (r"Date of decision\s+([\w\s]+),\s+(\d{4})\s+\(original decision\)", lambda m: m.group(1).strip().split() + [m.group(2).strip()]),
        "tribunal": (r"Tribunal\s*\n\s*([\w\s]+?)\s*\n\s*Date of decision", lambda m: m.group(1).replace(',', '').split()),
        "original": (r"Original\s+([\w]+\s+\d{1,2}(?:st|nd|rd|th)?,?\s+\d{4})", lambda m: m.group(1).replace(',', '').split())
    }

    for key, (pattern, process) in patterns.items():
        match = re.search(pattern, content, re.IGNORECASE)
        if match:
            return process(match)

    return None

## Date Formatter
Takes detected regular expression and converts into one common format

In [7]:
french_to_english = {
        'janvier': 'January', 'fevrier': 'February', 'mars': 'March', 'avril': 'April',
        'mai': 'May', 'juin': 'June', 'juillet': 'July', 'aout': 'August',
        'septembre': 'September', 'octobre': 'October', 'novembre': 'November', 'decembre': 'December'
}

def correct_month_name(misspelled_month, possibilities=['Janvier','January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December'], cutoff=0.6):
    correct_months = get_close_matches(misspelled_month, possibilities, n=1, cutoff=cutoff)
    if correct_months:
        corrected_month = correct_months[0]
        # Check if the corrected month is in the French to English mapping
        return french_to_english.get(corrected_month.lower(), corrected_month)
    else:
        return misspelled_month

def correct_year_typo(year):
    if len(year) == 3 and year.startswith("0"):
        return "20" + year[1:]
    return year

def correct_year_typo(year):
    """Corrects year format typos."""
    return "20" + year[1:] if len(year) == 3 and year.startswith("0") else year

def process_numeric_format(parts):
    """Processes numeric date format 'dd-mm-yyyy'."""
    day, month, year = parts[0].split('-')
    year = correct_year_typo(year)
    return datetime(int(year), int(month), int(day)).date().strftime('%Y-%m-%d')

def process_day_first_format(parts, french_month_mapping):
    """Processes dates in 'day month year' format, French or English."""
    day = 1 if parts[0].lower() == '1er' else int(parts[0])

    month = ''
    # Check if month and year are concatenated
    if len(parts) == 2 and not parts[1].isdigit():
        month_year_str = parts[1]
        for i in range(1, len(month_year_str)):
            if month_year_str[i:].isdigit():
                month_str, year_str = month_year_str[:i], month_year_str[i:]
                year = correct_year_typo(year_str)
                month = french_month_mapping.get(month_str.lower().replace('é', 'e').replace('û', 'u').replace('ô', 'o'), month_str.capitalize())
                break
    else:
        month = parts[1].lower().replace('é', 'e').replace('û', 'u').replace('ô', 'o')
        year = correct_year_typo(parts[2])

    if month in french_month_mapping:
        return datetime(int(year), french_month_mapping[month], day).date().strftime('%Y-%m-%d')
    else:
        if isinstance(month, int):
            return datetime(int(year), month, day).date().strftime('%Y-%m-%d')
        
        corrected_month = correct_month_name(month.capitalize())
        try:
            return datetime.strptime(f"{corrected_month} {day} {year}", '%B %d %Y').date().strftime('%Y-%m-%d')
        except ValueError as e:
            print(f"Error parsing date: {e}")
            return None

def process_month_first_format(parts):
    """Processes month first format with possible ordinal suffix."""
    day = 0
    month = ''
    year = ''
    
    if len(parts) == 2 and parts[1].isdigit() and len(parts[1]) > 2:
        
        if parts[1].isdigit() and len(parts[1]) > 4: 
            month = parts[0]
            year_str = parts[1][-4:]
            day_str = parts[1][:-4]
            year = year_str
            day = int(day_str)
            
        elif parts[1].isdigit()and len(parts[1]) > 3: #Year is the second entry
            month_day_str = parts[0]
            for i in range(1, len(month_day_str)):
                if not month_day_str[i].isdigit():
                    day_str, month_str = month_day_str[:i], month_day_str[i:]
                    day = int(day_str)
                    month = french_to_english.get(month_str.lower().replace('é', 'e').replace('û', 'u').replace('ô', 'o'), month_str)
                    parts[0] = month
                    break
            year = parts[1]
        else:
            year_str = parts[1][-4:]
            day_str = parts[1][:-4]
            year = correct_year_typo(year_str)
            day = int(day_str)

    else:
        day = re.sub(r"[^\d]", "", parts[1])
        day = int(day) if day.isdigit() else 1
        year = correct_year_typo(parts[2])
        
    
    try:
        corrected_month = correct_month_name(parts[0].capitalize())
        return datetime.strptime(f"{corrected_month} {day} {year}", '%B %d %Y').date().strftime('%Y-%m-%d')
    except ValueError as e:
        print(f"Error parsing date: {e}")
        return None


## Document Date Extraction
This function searches the given file for the document date using regular expressions, taking into account both French and English texts.

In [8]:
def process_date_parts(parts, french_month_mapping):
    """Determines the correct date processing method based on the format of the parts."""
    if '-' in parts[0]:
        return process_numeric_format(parts)
    elif parts[0].isdigit() or parts[0].lower() == '1er':
        return process_day_first_format(parts, french_month_mapping)
    else:
        return process_month_first_format(parts)

def extract_document_date(content):
    french_month_mapping = {
        'janvier': 1, 'fevrier': 2, 'mars': 3, 'avril': 4,
        'mai': 5, 'juin': 6, 'juillet': 7, 'aout': 8,
        'septembre': 9, 'octobre': 10, 'novembre': 11, 'decembre': 12
    }
    
    parts = match_date_patterns(content)
    
    if not parts:
        return None
    return process_date_parts(parts, french_month_mapping)

## File Processor Helpers

In [9]:
def extract_rad_number(content):
    """Extracts the RAD number from the content, ignoring IAD files."""
    # Check for lines indicating the file should be ignored
    ignore_lines = ["IAD File",
                    "IMMIGRATION APPEAL DIVISION", 
                    "ID File", 
                    "IMMIGRATION DIVISION", 
                    "RPD File", 
                    "RPD file",
                    "REFUGEE PROTECTION DIVISION", 
                    "REFUGEE DIVISION"
                    ]
    
    
    for line in content.splitlines():
     
        if any(ignore_line in line for ignore_line in ignore_lines):       
            return None
                
        sanitized_line = ''.join(c for c in line if c.isprintable()).strip()
    
        if "RAD File" in sanitized_line :
            
            rad_number_match = re.search(r"([A-Z]{2}\d+.\d+)", sanitized_line)
            if rad_number_match:
                return rad_number_match.group(1)
            
            # If RAD is in the next immediate line
            next_line_index = content.splitlines().index(line) + 1
            if next_line_index < len(content.splitlines()):
                next_line = content.splitlines()[next_line_index]
                rad_number_match = re.search(r"([A-Z]{2}\d+-\d+)", next_line)
                if rad_number_match:
                    return rad_number_match.group(1)
              
    return None



def process_file(file_path):
    """Processes a single file and extracts data."""
    with open(file_path, 'r', errors='replace') as file:
        content = file.read()

    rad_number = extract_rad_number(content)
    if rad_number:
        lang = detect_language(content)
        decision_maker_name = extract_decision_maker(content)
        document_date = extract_document_date(content)
        year = int(document_date.split('-')[0]) if document_date else None

        return {
            'citation1': rad_number,
            'citation2': '',
            'dataset': 'RAD',
            'name': '',
            'source_url': os.path.basename(file_path),
            #'scraped_timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
            'scraped_timestamp': datetime.fromtimestamp(os.path.getmtime(file_path)).strftime('%Y-%m-%d %H:%M:%S'),
            'document_date': document_date,
            'year': year,
            'unofficial_text': content,
            'language': lang,
            'other': json.dumps({'decision-maker_name': decision_maker_name}, ensure_ascii=False),
        }
    return None

## Processing Files
This block of code reads each file in the dataset directories to extract the needed information, using the previously defined functions and form a Pandas dataframe.

In [10]:
# Main data processing loop
data_records = []

for data_dir in DATA_DIRS:
    if os.path.exists(data_dir) and os.path.isdir(data_dir):
        for file_name in tqdm(os.listdir(data_dir)):
            #print(f"Parsing file: +{file_name}")
            if not file_name.startswith('~'):
                print(file_name)
                file_path = os.path.join(data_dir, file_name)
                record = process_file(file_path)
                if record:
                    data_records.append(record)

  0%|          | 0/3889 [00:00<?, ?it/s]

AppToReopenTB8-14702a.txt


  0%|          | 8/3889 [00:00<02:24, 26.88it/s]

AppToReopenTB8-14702tf.txt
Decision concerning G. Bazin (finale version) (amended 5.5.2022) (003) - sanitized.txt
Decision G. Bazin_ francais_modifiee (003) - sanitized.txt
MB7-00112a.txt
MB7-00112tf.txt
MB7-03926f.txt
MB7-03926ta.txt
MB7-24221 f.txt
MB7-24221ta.txt
MB8-01089a.txt
MB8-01089tf.txt
MB8-02514f.txt
MB8-02514ta.txt


  1%|          | 26/3889 [00:00<01:19, 48.55it/s]

...

977920.txt
977925.txt
978331.txt
978343.txt
978344.txt
978367.txt
979521.txt
979522.txt
979524.txt
979525.txt
979526.txt
979527.txt
979528.txt
979529.txt
979530.txt
979531.txt
979532.txt
979533.txt
979534.txt
979535.txt
979537.txt
979540.txt
979541.txt
979542.txt
979543.txt
979544.txt
979546.txt
979547.txt
979548.txt
979550.txt
979551.txt
979552.txt
979553.txt
979555.txt
979556.txt
979558.txt
979559.txt
979560.txt
979561.txt
979564.txt
979566.txt
979567.txt
979569.txt
979570.txt
979571.txt
979572.txt
979574.txt
979575.txt
979576.txt
979577.txt
979578.txt
979579.txt
979580.txt
979581.txt
979582.txt
979584.txt
979585.txt
979586.txt
979587.txt
979588.txt
979667.txt
979669.txt
979671.txt
979675.txt
979676.txt
979677.txt
979678.txt
979681.txt
979682.txt
979683.txt
979684.txt
979687.txt
979688.txt
979689.txt
979690.txt
979691.txt
979692.txt
979694.txt
979695.txt
979696.txt
979698.txt
979699.txt
979701.txt
979702.txt
979703.txt
979704.txt
979706.txt
979707.txt
979710.txt
979711.txt
979712.txt

100%|█████████▉| 116317/116688 [07:45<00:00, 2891.12it/s]

989621.txt
989622.txt
989623.txt
989625.txt
989626.txt
989628.txt
989629.txt
989631.txt
989633.txt
989634.txt
989637.txt
989638.txt
989639.txt
989642.txt
989643.txt
989644.txt
989646.txt
989648.txt
989649.txt
989650.txt
989651.txt
989653.txt
989654.txt
989655.txt
989656.txt
989659.txt
989660.txt
989661.txt
989664.txt
989666.txt
989668.txt
989669.txt
989670.txt
989671.txt
989673.txt
989675.txt
989676.txt
989677.txt
989678.txt
989679.txt
989680.txt
989681.txt
989682.txt
989683.txt
989684.txt
989685.txt
990366.txt
990367.txt
990373.txt
990377.txt
990379.txt
990381.txt
990382.txt
990383.txt
990387.txt
990388.txt
990389.txt
990390.txt
990392.txt
990393.txt
990394.txt
990395.txt
990396.txt
990398.txt
990403.txt
990406.txt
990407.txt
990408.txt
990409.txt
990410.txt
990416.txt
990417.txt
990419.txt
990420.txt
990422.txt
990423.txt
990424.txt
990426.txt
990427.txt
990428.txt
990429.txt
990430.txt
990435.txt
990438.txt
990439.txt
990441.txt
990443.txt
990446.txt
990448.txt
990449.txt
990450.txt

100%|██████████| 116688/116688 [07:45<00:00, 250.41it/s] 

996777.txt
996778.txt
996779.txt
996782.txt
996783.txt
996785.txt
996787.txt
996789.txt
996790.txt
996791.txt
996792.txt
996793.txt
996797.txt
996798.txt
996799.txt
996800.txt
996803.txt
996806.txt
996807.txt
996808.txt
996809.txt
997262.txt
997263.txt
997265.txt
997266.txt
997267.txt
997268.txt
997269.txt
997271.txt
997272.txt
997275.txt
997278.txt
997280.txt
997284.txt
997285.txt
997286.txt
997288.txt
997291.txt
997294.txt
997296.txt
997298.txt
997300.txt
997301.txt
997304.txt
997305.txt
997306.txt
997308.txt
997309.txt
997310.txt
997312.txt
997313.txt
997315.txt
997316.txt
997318.txt
997320.txt
997322.txt
997325.txt
997326.txt
997327.txt
997328.txt
997330.txt
997332.txt
997333.txt
997336.txt
997337.txt
997339.txt
997340.txt
997342.txt
997343.txt
997344.txt
997345.txt
997348.txt
997350.txt
997352.txt
997353.txt
997921.txt
997922.txt
997924.txt
997926.txt
997927.txt
997929.txt
997931.txt
997933.txt
997934.txt
997936.txt
997937.txt
997938.txt
997939.txt
997941.txt
997942.txt
997947.txt

## Converting to DF and data cleaning
Converts into dataframe and cleans data

In [11]:
# convert to dataframe
df = pd.DataFrame(data_records)

# fix dates format
df['document_date'] = pd.to_datetime(df['document_date']).dt.strftime('%Y-%m-%d')
df['scraped_timestamp'] = pd.to_datetime(df['scraped_timestamp']).dt.strftime('%Y-%m-%d')

print (len(df))
# drop where year is nan
df = df.dropna(subset=['year'])
print (len(df))
# convert year to int
df['year'] = df['year'].astype(int)

# Remove rows where unofficial text is duplicated, keeping the last one
df = df.drop_duplicates(subset=['unofficial_text'], keep='last')
print (len(df))
#df



27996
27675
27246


### Exports to JSON

In [13]:
# get start and end year
start_year = df['year'].min()
end_year = df['year'].max()

# set output dir
output_dir = 'DATA/YEARLY'

# export cleaned df to yearly / language json files
for year in tqdm(range(start_year, end_year+1)):
    for language in ['en', 'fr']:
        out_path_yearly_lang = output_dir + "/"+ f'{year}_{language}.json'
        df[(df.year == year) & (df.language == language)].to_json(out_path_yearly_lang, orient='records', indent=4)

100%|██████████| 14/14 [00:12<00:00,  1.16it/s]
